In [4]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions # this package provides a lot of nice abstractions for policy gradients
from torch.autograd import Variable
from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors
# import spacy
# import matplotlib.pyplot as plt
# import matplotlib.ticker as ticker

import csv
import time
import argparse

# You can add MIN_FREQ, MAX_LEN, and BATCH_SIZE as args too

model_type = args.model_type
n_epochs = args.n_epochs
learning_rate = args.learning_rate
attn_type = args.attn_type
clip_constraint = args.clip_constraint
word2vec = args.word2vec
rho = args.rho
weight_decay = args.weight_decay

spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

BOS_WORD = '<s>'
EOS_WORD = '</s>'
DE = data.Field(tokenize=tokenize_de)
EN = data.Field(tokenize=tokenize_en, init_token = BOS_WORD, eos_token = EOS_WORD) # only target needs BOS/EOS

print("Getting datasets!")

MAX_LEN = 20
train, val, test = datasets.IWSLT.splits(exts=('.de', '.en'), fields=(DE, EN), 
                                         filter_pred=lambda x: len(vars(x)['src']) <= MAX_LEN and 
                                         len(vars(x)['trg']) <= MAX_LEN)

print(train.fields)
print(len(train))
print(vars(train[0]))

MIN_FREQ = 5
DE.build_vocab(train.src, min_freq=MIN_FREQ)
EN.build_vocab(train.trg, min_freq=MIN_FREQ)
print(DE.vocab.freqs.most_common(10))
print("Size of German vocab", len(DE.vocab))
print(EN.vocab.freqs.most_common(10))
print("Size of English vocab", len(EN.vocab))
print(EN.vocab.stoi["<s>"], EN.vocab.stoi["</s>"]) # vocab index for <s>, </s>

BATCH_SIZE = 32
train_iter, val_iter = data.BucketIterator.splits((train, val), batch_size=BATCH_SIZE, device=-1,
                                                  repeat=False, sort_key=lambda x: len(x.src))

batch = next(iter(train_iter))
print("Source size", batch.src.size())
print("Target size", batch.trg.size())

# https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md
if word2vec:
    url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
    EN.vocab.load_vectors(vectors=Vectors('wiki.simple.vec', url=url)) # feel free to alter path
    print("Simple English embeddings size", EN.vocab.vectors.size())
    url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.de.vec'
    DE.vocab.load_vectors(vectors=Vectors('wiki.de.vec', url=url)) # feel free to alter path
    print("German embeddings size", DE.vocab.vectors.size())

print("REMINDER!!! Did you create ../../models/HW3?????")

sos_token = EN.vocab.stoi["<s>"]
eos_token = EN.vocab.stoi["</s>"]
pad_token = EN.vocab.stoi["<pad>"]

NameError: name 'args' is not defined

In [2]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

In [16]:
def visualize(attns,sentence_de,wordlist):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(np.array(attns), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels(sentence_de, rotation=90)
    ax.set_yticklabels(wordlist)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

In [17]:
attns=[[0.02,0.03,0.04,0.01,0.0001,0.001],
       [0.04,0.02,0.04,0.004,0.0114,0.005],
       [0.04,0.02,0.04,0.004,0.0114,0.005],
       [0.04,0.02,0.04,0.004,0.0114,0.005],
       [0.04,0.02,0.04,0.004,0.0114,0.005],
       [0.04,0.02,0.04,0.004,0.0114,0.005]]
sentence_de = ["german1","german2","german3","german4","german5","german6"]
wordlist = ["english1","english2","english3","english4","english5","english6"]

In [18]:
visualize(attns,sentence_de,wordlist)

In [ ]:
2+3

In [3]:
fig = plt.figure()

In [4]:
ax = fig.add_subplot(111)

In [7]:
cax = ax.matshow(np.array(attns), cmap='bone')

In [8]:
fig.colorbar(cax)

In [9]:
ax.set_xticklabels(sentence_de, rotation=90)

In [10]:
ax.set_yticklabels(wordlist)

In [11]:
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))

In [12]:
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

In [13]:
plt.show()

In [1]:
en_wordlist = [[175,0,9,6,0,9,6,0,141,16],
 [0,9,6,0,9,141,16,141,16,141],
 [175,0,9,6,0,9,6,0,32,16],
 [0,9,6,0,9,141,16,141,16,0],
 [175,0,9,6,0,9,6,0,141,0]]